In [0]:
-- Customers Silver (deduplicate + drop invalid custkey)
CREATE OR REFRESH MATERIALIZED VIEW silver.customers_silver(
CONSTRAINT valid_custkey EXPECT (c_custkey IS NOT NULL) ON VIOLATION DROP ROW)
AS
SELECT DISTINCT *
FROM (bronze.customers);

-------------------------------------------------------------

-- Orders Silver (drop invalid orderdate, negative price, null custkey)
CREATE OR REFRESH MATERIALIZED VIEW silver.orders_silver(
CONSTRAINT valid_orderdate EXPECT (o_orderdate IS NOT NULL) ON VIOLATION DROP ROW,
CONSTRAINT valid_totalprice EXPECT (o_totalprice >= 0) ON VIOLATION DROP ROW,
CONSTRAINT valid_custkey EXPECT (o_custkey IS NOT NULL) ON VIOLATION DROP ROW)
AS
SELECT *
FROM (bronze.orders);

-------------------------------------------------------------

-- Nation Silver (drop missing nation or region keys)
CREATE OR REFRESH STREAMING TABLE silver.nation_silver(
CONSTRAINT valid_nationkey EXPECT (n_nationkey IS NOT NULL) ON VIOLATION DROP ROW,
CONSTRAINT valid_regionkey EXPECT (n_regionkey IS NOT NULL) ON VIOLATION DROP ROW)
AS
SELECT DISTINCT *
FROM STREAM(bronze.nation);

-------------------------------------------------------------

-- Region Silver (drop missing region key)
CREATE OR REFRESH MATERIALIZED VIEW silver.region_silver(
CONSTRAINT valid_regionkey EXPECT (r_regionkey IS NOT NULL) ON VIOLATION DROP ROW)
AS
SELECT DISTINCT *
FROM (bronze.region);